In [1]:
import gymnasium as gym
from gymnasium import spaces
import pygame
import numpy as np

class ScreenSaverEnv(gym.Env):
    def __init__(self, canvas_size=(800, 600), image_path="path_to_image.png", speed=5):
        super(ScreenSaverEnv, self).__init__()
        
        # Initialize canvas and image properties
        self.canvas_width, self.canvas_height = canvas_size
        self.image = pygame.image.load(image_path)
        # Resize image to fit within the canvas
        resize = (100, 100)
        # Apply the resize to the image
        self.image = pygame.transform.scale(self.image, resize)
        self.image_width, self.image_height = self.image.get_size()
        
        # Initialize starting position and velocity
        self.position = np.array([np.random.randint(0, self.canvas_width - self.image_width), 
                                  np.random.randint(0, self.canvas_height - self.image_height)])
        self.velocity = np.array([speed, speed])
        
        # Action space: No action needed, movement is automated
        self.action_space = spaces.Discrete(1)  # Dummy action space
        # Observation space: RGB array of the canvas
        self.observation_space = spaces.Box(low=0, high=255, shape=(self.canvas_height, self.canvas_width, 3), dtype=np.uint8)
        
        # Initialize pygame
        pygame.init()
        self.screen = pygame.display.set_mode(canvas_size)
        pygame.display.set_caption("Screensaver Environment")

    def reset(self):
        # Reset position to a random location
        self.position = np.array([np.random.randint(0, self.canvas_width - self.image_width), 
                                  np.random.randint(0, self.canvas_height - self.image_height)])
        # Reset velocity
        self.velocity = np.array([np.random.choice([-1, 1]), np.random.choice([-1, 1])]) * np.linalg.norm(self.velocity)
        
        return self._get_observation(), {}

    def step(self, action):
        # Update position
        self.apply_velocity(action)
        
        # Bounce off the walls
        if self.position[0] <= 0 or self.position[0] + self.image_width >= self.canvas_width:
            self.velocity[0] = -self.velocity[0]
        if self.position[1] <= 0 or self.position[1] + self.image_height >= self.canvas_height:
            self.velocity[1] = -self.velocity[1]
        
        # No specific reward; observation only
        return self._get_observation(), 0, False, False, {}

    # Method that apply the velocity to the position with bounce
    def apply_velocity(self, action):
        # Update position
        # Check if the position is out of the screen
        if self.position[0] <= 0 or self.position[0] + self.image_width >= self.canvas_width:
            self.velocity[0] = -self.velocity[0]
        if self.position[1] <= 0 or self.position[1] + self.image_height >= self.canvas_height:
            self.velocity[1] = -self.velocity[1]
        # Update position
        self.position.__add__(self.velocity)


    def _get_observation(self):
        # Render the current frame as an observation
        self.screen.fill((0, 0, 0))  # Clear screen with black
        self.screen.blit(self.image, self.position)  # Draw image at current position
        pygame.display.flip()  # Update display
        
        # Convert the pygame screen to a numpy array
        frame = pygame.surfarray.array3d(self.screen)
        return np.transpose(frame, (1, 0, 2))  # Transpose to match Gym's (H, W, C) format

    def render(self, mode="human"):
        if mode == "human":
            pygame.display.flip()

    def close(self):
        pygame.quit()

# Example usage
env = ScreenSaverEnv(canvas_size=(800, 600), image_path="lebronpng.png", speed=5)
obs, info = env.reset()
done = False
while not done:
    obs, reward, done, truncated, info = env.step(0)
    env.render()

KeyboardInterrupt: 